In [70]:
# python libraries
import os
import sys
import dataclasses
from datetime import datetime
from pathlib import Path
from operator import methodcaller
from collections import OrderedDict
from dataclasses import dataclass
from typing import (
    List,
    Tuple,
    Dict,
    Any,
    Mapping,
    Callable
)
from enum import Enum
# adding the path
if not str(Path(os.getcwd()).parent) in sys.path:
    sys.path.append(str(Path(os.getcwd()).parent))

# numpy
import numpy as np

# torch
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models

try:
    from torchmetrics import Accuracy
except:
    print(f"[INFO] Installing the torchmetrics")
    %pip install torchmetrics
    from torchmetrics import Accuracy

try:
    import torchinfo
except:
    print(f"[INFO] Installing the torchinfo")
    %pip install torchinfo
    import torchinfo

# helper function
try:
    import my_helper as helper
except:
    print("[INFO] Downloading the helper function from github")
    import requests
    response = requests.get("https://raw.githubusercontent.com/Lashi0812/PyTorch2/master/my_helper.py")
    with open("my_helper.py" ,"wb") as f:
        f.write(response.content)
    import my_helper as helper


## Connect Persistence memory
try :
    from google.colab import drive

    # Paths
    DRIVE_PATH = Path("/content/drive")
    MODEL_SAVE_PATH = Path("/content/drive/Othercomputers/My PC/drive/models")

    # mount drive
    drive.mount(str(DRIVE_PATH))
except:
    MODEL_SAVE_PATH = Path(os.getcwd())/"models"
    
device = "cuda" if torch.cuda.is_available() else "cpu"

# Conv + BN 

In [77]:
class BasicConv2d(nn.Module):
    def __init__(self,out_channels,**kwargs) -> None:
        super().__init__()
        self.bc = nn.Sequential(
            nn.LazyConv2d(out_channels,bias=False,**kwargs),
            nn.LazyBatchNorm2d()
        )
    def forward(self,x):
        return self.bc(x)

# Residual Block

In [78]:
class ResidualBlock(nn.Module):
    def __init__(self,out_channels,stride=1,use_conv1x1=False) -> None:
        super().__init__()
        self.bc1 = BasicConv2d(out_channels,kernel_size=3,stride=stride,padding=1)
        self.bc2 = BasicConv2d(out_channels,kernel_size=3,stride=1,padding=1)
        if use_conv1x1:
            self.conv1x1 = BasicConv2d(out_channels,kernel_size=1,padding=0,stride=stride)
        else:
            self.conv1x1 = None
        
    def forward(self,x):
        y = F.relu(self.bc1(x))
        y = self.bc2(y)
        if self.conv1x1:
            x = self.conv1x1(x)
        y += x
        return F.relu(y)      

In [85]:
class ResNet(nn.Module):
    def __init__(self,arch,num_classes=1000) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.net = nn.Sequential(self.b1())
        for i,b in enumerate(arch):
            self.net.add_module(f"{i+2}",self.block(*b,first_block=(i==0)))
        self.net.add_module("last",nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),nn.Flatten(),
            nn.LazyLinear(num_classes)
        ))
    
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64,kernel_size=7,stride=2,padding=3,bias=False),
            nn.LazyBatchNorm2d(),nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1)           
        )
        
    def block(self,num_residual,out_channels,first_block=False):
        blk = []
        for i in range(num_residual):
            if i == 0 and not first_block:
                blk.append(ResidualBlock(out_channels,stride=2,use_conv1x1=True))
            else:
                blk.append(ResidualBlock(out_channels,use_conv1x1=False))
        return nn.Sequential(*blk)
    
    def forward(self,x):
        return self.net(x)


# ResNet18

In [86]:
class ResNet18(ResNet):
    def __init__(self, num_classes=1000) -> None:
        super().__init__(((2,64),(2,128),(2,256),(2,512)), num_classes)

In [87]:
torchinfo.summary(ResNet18(),input_size=(1,3,224,224)  ,  col_width=14,depth=3,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

c:\Users\Lenovo\anaconda3\envs\py310\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Layer (type (var_name))                            Kernel Shape   Output Shape   Param #        Mult-Adds
ResNet18 (ResNet18)                                --             [1, 1000]      --             --
├─Sequential (net)                                 --             [1, 1000]      --             --
│    └─Sequential (0)                              --             [1, 64, 56, 56] --             --
│    │    └─Conv2d (0)                             [7, 7]         [1, 64, 112, 112] 9,408          118,013,952
│    │    └─BatchNorm2d (1)                        --             [1, 64, 112, 112] 128            128
│    │    └─ReLU (2)                               --             [1, 64, 112, 112] --             --
│    │    └─MaxPool2d (3)                          3              [1, 64, 56, 56] --             --
│    └─Sequential (2)                              --             [1, 64, 56, 56] --             --
│    │    └─ResidualBlock (0)                      --             [1, 64, 56, 56

In [88]:
torchinfo.summary(models.resnet18(),input_size=(1,3,224,224)  ,  col_width=14,depth=2,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                  Kernel Shape   Output Shape   Param #        Mult-Adds
ResNet (ResNet)                          --             [1, 1000]      --             --
├─Conv2d (conv1)                         [7, 7]         [1, 64, 112, 112] 9,408          118,013,952
├─BatchNorm2d (bn1)                      --             [1, 64, 112, 112] 128            128
├─ReLU (relu)                            --             [1, 64, 112, 112] --             --
├─MaxPool2d (maxpool)                    3              [1, 64, 56, 56] --             --
├─Sequential (layer1)                    --             [1, 64, 56, 56] --             --
│    └─BasicBlock (0)                    --             [1, 64, 56, 56] 73,984         231,211,264
│    └─BasicBlock (1)                    --             [1, 64, 56, 56] 73,984         231,211,264
├─Sequential (layer2)                    --             [1, 128, 28, 28] --             --
│    └─BasicBlock (0)                    --             [1, 

# ResNet34

In [89]:
class ResNet34(ResNet):
    def __init__(self, num_classes=1000) -> None:
        super().__init__(((3,64),(4,128),(6,256),(3,512)), num_classes)
        

In [92]:
torchinfo.summary(ResNet34(),input_size=(1,3,224,224)  ,  col_width=12,depth=3,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                            Kernel Shape Output Shape Param #      Mult-Adds
ResNet34 (ResNet34)                                --           [1, 1000]    --           --
├─Sequential (net)                                 --           [1, 1000]    --           --
│    └─Sequential (0)                              --           [1, 64, 56, 56] --           --
│    │    └─Conv2d (0)                             [7, 7]       [1, 64, 112, 112] 9,408        118,013,952
│    │    └─BatchNorm2d (1)                        --           [1, 64, 112, 112] 128          128
│    │    └─ReLU (2)                               --           [1, 64, 112, 112] --           --
│    │    └─MaxPool2d (3)                          3            [1, 64, 56, 56] --           --
│    └─Sequential (2)                              --           [1, 64, 56, 56] --           --
│    │    └─ResidualBlock (0)                      --           [1, 64, 56, 56] 73,984       231,211,264
│    │    └─Resid

In [224]:
torchinfo.summary(models.resnet34(),input_size=(1,3,224,224)  ,  col_width=14,depth=3,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                  Kernel Shape   Output Shape   Param #        Mult-Adds
ResNet (ResNet)                          --             [1, 1000]      --             --
├─Conv2d (conv1)                         [7, 7]         [1, 64, 112, 112] 9,408          118,013,952
├─BatchNorm2d (bn1)                      --             [1, 64, 112, 112] 128            128
├─ReLU (relu)                            --             [1, 64, 112, 112] --             --
├─MaxPool2d (maxpool)                    3              [1, 64, 56, 56] --             --
├─Sequential (layer1)                    --             [1, 64, 56, 56] --             --
│    └─BasicBlock (0)                    --             [1, 64, 56, 56] --             --
│    │    └─Conv2d (conv1)               [3, 3]         [1, 64, 56, 56] 36,864         115,605,504
│    │    └─BatchNorm2d (bn1)            --             [1, 64, 56, 56] 128            128
│    │    └─ReLU (relu)                  --             [1, 64, 56, 5

# ReducedResidualBlock

In [282]:
class ReducedResidualBlock(nn.Module):
    def __init__(self,out_channels,use_conv1x1=False,stride=1) -> None:
        super().__init__()
        self.out_channels = out_channels
        self.bc1 = BasicConv2d(out_channels[0],kernel_size=1,stride=1,padding=0)
        self.bc2 = BasicConv2d(out_channels[0],kernel_size=3,stride=stride,padding=1)
        self.bc3 = BasicConv2d(out_channels[1],kernel_size=1,stride=1,padding=0)
        if use_conv1x1:
            self.conv1x1 = BasicConv2d(out_channels[1],kernel_size=1,stride=stride,padding=0)
        else:
            self.conv1x1 = None
    
    def forward(self,x):
        y = F.relu(self.bc1(x))
        y = F.relu(self.bc2(y))
        y = self.bc3(y)
        if self.conv1x1:
            x = self.conv1x1(x)
        y += x 
        
        return F.relu(y)

# Modify ResNet

In [283]:
class ResNet(nn.Module):
    def __init__(self,arch,residual_block,num_classes=1000) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.net = nn.Sequential(self.b1())
        self.residual_block = residual_block
        for i,b in enumerate(arch):
            self.net.add_module(f"{i+2}",self.block(*b,first_block=(i==0)))
        self.net.add_module("last",nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),nn.Flatten(),
            nn.LazyLinear(num_classes)
        ))
    
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64,kernel_size=7,stride=2,padding=3,bias=False),
            nn.LazyBatchNorm2d(),nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1)           
        )
        
    def block(self,num_residual,out_channels,first_block=False):
        blk = []
        for i in range(num_residual):
            if first_block:
                if i ==0:
                    blk.append(self.residual_block(out_channels,use_conv1x1=True,stride=1))
                else :
                    blk.append(self.residual_block(out_channels,use_conv1x1=False,stride=1))
            else:
                if i == 0:
                    blk.append(self.residual_block(out_channels=out_channels,stride=2,use_conv1x1=True))
                else:
                    blk.append(self.residual_block(out_channels=out_channels,use_conv1x1=False))
        return nn.Sequential(*blk)
    
    def forward(self,x):
        return self.net(x)


In [285]:
class Resnet50(ResNet):
    def __init__(self, num_classes=1000) -> None:
        super().__init__(arch=((3,(64,256)),(4,(128,512)),(6,(256,1024)),(3,(512,2048))),
                         residual_block= ReducedResidualBlock,
                         num_classes= num_classes)

In [287]:
torchinfo.summary(Resnet50(),input_size=(1,3,224,224),col_width=12,depth=3,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                            Kernel Shape Output Shape Param #      Mult-Adds
Resnet50 (Resnet50)                                --           [1, 1000]    --           --
├─Sequential (net)                                 --           [1, 1000]    --           --
│    └─Sequential (0)                              --           [1, 64, 56, 56] --           --
│    │    └─Conv2d (0)                             [7, 7]       [1, 64, 112, 112] 9,408        118,013,952
│    │    └─BatchNorm2d (1)                        --           [1, 64, 112, 112] 128          128
│    │    └─ReLU (2)                               --           [1, 64, 112, 112] --           --
│    │    └─MaxPool2d (3)                          3            [1, 64, 56, 56] --           --
│    └─Sequential (2)                              --           [1, 256, 56, 56] --           --
│    │    └─ReducedResidualBlock (0)               --           [1, 256, 56, 56] 75,008       231,212,288
│    │    └─Red

In [289]:
torchinfo.summary(models.resnet50(),input_size=(1,3,224,224)  ,  col_width=14,depth=2,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                  Kernel Shape   Output Shape   Param #        Mult-Adds
ResNet (ResNet)                          --             [1, 1000]      --             --
├─Conv2d (conv1)                         [7, 7]         [1, 64, 112, 112] 9,408          118,013,952
├─BatchNorm2d (bn1)                      --             [1, 64, 112, 112] 128            128
├─ReLU (relu)                            --             [1, 64, 112, 112] --             --
├─MaxPool2d (maxpool)                    3              [1, 64, 56, 56] --             --
├─Sequential (layer1)                    --             [1, 256, 56, 56] --             --
│    └─Bottleneck (0)                    --             [1, 256, 56, 56] 75,008         231,212,288
│    └─Bottleneck (1)                    --             [1, 256, 56, 56] 70,400         218,366,720
│    └─Bottleneck (2)                    --             [1, 256, 56, 56] 70,400         218,366,720
├─Sequential (layer2)                    --     

## ResNet 101

In [290]:
class Resnet101(ResNet):
    def __init__(self, num_classes=1000) -> None:
        super().__init__(arch=((3,(64,256)),(4,(128,512)),(23,(256,1024)),(3,(512,2048))),
                         residual_block= ReducedResidualBlock,
                         num_classes= num_classes)

In [291]:
torchinfo.summary(Resnet101(),input_size=(1,3,224,224)  ,  col_width=12,depth=2,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                            Kernel Shape Output Shape Param #      Mult-Adds
Resnet101 (Resnet101)                              --           [1, 1000]    --           --
├─Sequential (net)                                 --           [1, 1000]    --           --
│    └─Sequential (0)                              --           [1, 64, 56, 56] 9,536        118,014,080
│    └─Sequential (2)                              --           [1, 256, 56, 56] 215,808      667,945,728
│    └─Sequential (3)                              --           [1, 512, 28, 28] 1,219,584    1,027,611,648
│    └─Sequential (4)                              --           [1, 1024, 14, 14] 26,090,496   5,176,630,272
│    └─Sequential (5)                              --           [1, 2048, 7, 7] 14,964,736   809,261,056
│    └─Sequential (last)                           --           [1, 1000]    2,049,000    2,049,000
Total params: 44,549,160
Trainable params: 44,549,160
Non-trainable params: 0
Tot

In [292]:
torchinfo.summary(models.resnet101(),input_size=(1,3,224,224)  ,  col_width=14,depth=2,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                  Kernel Shape   Output Shape   Param #        Mult-Adds
ResNet (ResNet)                          --             [1, 1000]      --             --
├─Conv2d (conv1)                         [7, 7]         [1, 64, 112, 112] 9,408          118,013,952
├─BatchNorm2d (bn1)                      --             [1, 64, 112, 112] 128            128
├─ReLU (relu)                            --             [1, 64, 112, 112] --             --
├─MaxPool2d (maxpool)                    3              [1, 64, 56, 56] --             --
├─Sequential (layer1)                    --             [1, 256, 56, 56] --             --
│    └─Bottleneck (0)                    --             [1, 256, 56, 56] 75,008         231,212,288
│    └─Bottleneck (1)                    --             [1, 256, 56, 56] 70,400         218,366,720
│    └─Bottleneck (2)                    --             [1, 256, 56, 56] 70,400         218,366,720
├─Sequential (layer2)                    --     

# ResNet 152

In [293]:
class Resnet152(ResNet):
    def __init__(self, num_classes=1000) -> None:
        super().__init__(arch=((3,(64,256)),(8,(128,512)),(36,(256,1024)),(3,(512,2048))),
                         residual_block= ReducedResidualBlock,
                         num_classes= num_classes)

In [295]:
torchinfo.summary(Resnet152(),input_size=(1,3,224,224)  ,  col_width=12,depth=3,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                            Kernel Shape Output Shape Param #      Mult-Adds
Resnet152 (Resnet152)                              --           [1, 1000]    --           --
├─Sequential (net)                                 --           [1, 1000]    --           --
│    └─Sequential (0)                              --           [1, 64, 56, 56] --           --
│    │    └─Conv2d (0)                             [7, 7]       [1, 64, 112, 112] 9,408        118,013,952
│    │    └─BatchNorm2d (1)                        --           [1, 64, 112, 112] 128          128
│    │    └─ReLU (2)                               --           [1, 64, 112, 112] --           --
│    │    └─MaxPool2d (3)                          3            [1, 64, 56, 56] --           --
│    └─Sequential (2)                              --           [1, 256, 56, 56] --           --
│    │    └─ReducedResidualBlock (0)               --           [1, 256, 56, 56] 75,008       231,212,288
│    │    └─Red

In [297]:
torchinfo.summary(models.resnet152(),input_size=(1,3,224,224)  ,  col_width=14,depth=2,
    col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
    row_settings=["var_names"],
)

Layer (type (var_name))                  Kernel Shape   Output Shape   Param #        Mult-Adds
ResNet (ResNet)                          --             [1, 1000]      --             --
├─Conv2d (conv1)                         [7, 7]         [1, 64, 112, 112] 9,408          118,013,952
├─BatchNorm2d (bn1)                      --             [1, 64, 112, 112] 128            128
├─ReLU (relu)                            --             [1, 64, 112, 112] --             --
├─MaxPool2d (maxpool)                    3              [1, 64, 56, 56] --             --
├─Sequential (layer1)                    --             [1, 256, 56, 56] --             --
│    └─Bottleneck (0)                    --             [1, 256, 56, 56] 75,008         231,212,288
│    └─Bottleneck (1)                    --             [1, 256, 56, 56] 70,400         218,366,720
│    └─Bottleneck (2)                    --             [1, 256, 56, 56] 70,400         218,366,720
├─Sequential (layer2)                    --     